In [1]:
from Tight_Binding import extract_parameter

In [2]:
hr_files = "First_Principle/Sr10_Nb10_O34_hr.dat"

In [6]:
cls_parameter = extract_parameter(hr_files)
parameter = cls_parameter.get_data()
parameter.head()

,X,Y,Z,A,B,Re,Im
0,-2,0,-4,1,1,-0.000026,-0.000008
1,-2,0,-4,2,1,0.000097,-0.000160
2,-2,0,-4,3,1,-0.000066,0.000386
3,-2,0,-4,4,1,-0.000022,-0.000037
4,-2,0,-4,5,1,0.000124,-0.000033


In [7]:
parameter.tail()

,X,Y,Z,A,B,Re,Im
5387215,2,0,4,342,6,0.000060,0.000121
5387216,2,0,4,343,6,0.000079,-0.000039
5387217,2,0,4,344,6,-0.000027,0.000005
5387218,2,0,4,345,6,0.000281,-0.000018
5387219,2,0,4,346,6,-0.000171,-0.000006


In [8]:
parameter.describe()

,X,Y,Z,A,B,Re,Im
count,5.387220e+06,5387220.0,5.387220e+06,5.387220e+06,5.387220e+06,5.387220e+06,5.387220e+06
mean,0.000000e+00,0.0,0.000000e+00,1.735000e+02,4.482659e+00,3.376828e-04,1.036758e-04
std,1.414214e+00,0.0,2.581989e+00,9.988119e+01,2.859118e+00,9.929295e-02,7.606113e-02
min,-2.000000e+00,0.0,-4.000000e+00,1.000000e+00,0.000000e+00,-9.880025e+00,-9.934783e+00
25%,-1.000000e+00,0.0,-2.000000e+00,8.700000e+01,2.000000e+00,-1.350000e-04,-1.220000e-04
50%,0.000000e+00,0.0,0.000000e+00,1.735000e+02,4.000000e+00,-0.000000e+00,0.000000e+00
75%,1.000000e+00,0.0,2.000000e+00,2.600000e+02,7.000000e+00,1.350000e-04,1.220000e-04
max,2.000000e+00,0.0,4.000000e+00,3.460000e+02,9.000000e+00,2.065871e+01,1.448322e+01
